## Final Project Submission

Please fill out:
* Student name: Abdulrahman Aber, Benjamin Bai, Charles Pan, Kevin Rivera
* Student pace: Full Time
* Scheduled project review date/time: 
* Instructor name: David Elliott
* Blog post URL:


# Imports, Data Frame Initializations

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sqlite3 as sq
import seaborn as sns
import statistics as st
import datetime
conn = sq.connect('./zippedData/im.db')
cur = conn.cursor()

In [4]:
movie_basics_df = pd.read_sql('''select * from movie_basics''', conn)

In [ ]:
movie_ratings_df = pd.read_sql("""select * from movie_ratings""",conn)

In [ ]:
persons_df = pd.read_sql("""select * from persons""",conn)

In [ ]:
principals_df = pd.read_sql("""select * from principals""",conn)

In [ ]:
tmdb_movies_df = pd.read_csv('./zippedData/tmdb.movies.csv.gz')

In [ ]:
tn_movie_budgets_df = pd.read_csv('./zippedData/tn.movie_budgets.csv.gz',
                                 parse_dates=['release_date'])

In [ ]:
tn_movie_budgets_df.info()

tn_movie_budgets_df.info() shows us that this dataframe will be used to derive net_return and ROI as a percentage, specifically production_budget and worldwide_gross. We believe net_return and ROI% are the key metrics for an executive team weighing different investment options. 

# TN Movie Budgets Clean Up Process (Genres Analysis)

In [ ]:
# Remove $ from production_budget, convert to float
tn_movie_budgets_df['production_budget'] = tn_movie_budgets_df['production_budget'].str.replace(',', '').str.replace('$', '').astype(float)

# Remove $ from domestic_gross, convert to float
tn_movie_budgets_df['domestic_gross'] = tn_movie_budgets_df['domestic_gross'].str.replace(',', '').str.replace('$', '').astype(float)

# Remove $ from worldwide_gross, convert to float
tn_movie_budgets_df['worldwide_gross'] = tn_movie_budgets_df['worldwide_gross'].str.replace(',', '').str.replace('$', '').astype(float)

# new column release_year from release_date to filter based on time frame in scope
tn_movie_budgets_df['release_year'] = pd.DatetimeIndex(tn_movie_budgets_df['release_date']).year

# time frame: 2010-Present; SAG definition of theatrical budget: > 2 million
tn_movie_budgets_df = tn_movie_budgets_df[(tn_movie_budgets_df.release_year >= 2010) & (tn_movie_budgets_df.production_budget > 1999999)]

# Recommendation 1: Genres Analysis
TN Movie Budgets merge with IMDB Movie Basics to compare net return and ROI percent

In [ ]:
# rename movie_basics column to match tn_movie_budgets
movie_basics_df.rename(columns={"primary_title":"movie"}, inplace = True)

# merge movie_budgets and imdb movie_basics
moviebudgets_moviebasics_df = tn_movie_budgets_df.merge(movie_basics_df, how='inner', on='movie')

# drop columns that are not relevant to Genre analysis
moviebudgets_moviebasics_df.drop(columns = ['id', 'movie_id', 'original_title', 'runtime_minutes', 'domestic_gross' ],axis=1,inplace=True)

# remove duplicate movies
moviebudgets_moviebasics_df.drop_duplicates(subset='movie', keep='last', inplace=True, ignore_index=False)

# add column for net_return
moviebudgets_moviebasics_df['net_return'] = (moviebudgets_moviebasics_df['worldwide_gross'] - moviebudgets_moviebasics_df['production_budget'])

# add column for ROI as a %
moviebudgets_moviebasics_df['roi_percent'] = ((moviebudgets_moviebasics_df['net_return'] / moviebudgets_moviebasics_df['production_budget'])*100)

In [ ]:
moviebudgets_moviebasics_df.info()

Our sample size is 3733 Unique Films since 2010 to perform Genres Analysis on and after the cleanup process, we now have all the data we need to compare net_return and ROI percent across these films to best inform our recommendation for the executive team at Microsoft

# Recommendation 1: Visualizations

In [ ]:
# genres to list 
moviebudgets_moviebasics_df['genres'] = moviebudgets_moviebasics_df['genres'].str.split(",")
# explode out rows for movies with multiple genres for visualization purposes
moviebudgets_moviebasics_df = moviebudgets_moviebasics_df.explode('genres')
# means grouped by genre
moviebudgets_moviebasics_mean_df = moviebudgets_moviebasics_df.groupby(['genres']).mean().reset_index()

In [ ]:
# visualization 1: initialize relevant dataframes and lists for x/y values
netreturn_vis = moviebudgets_moviebasics_df.dropna()

n_list = list(moviebudgets_moviebasics_df['genres'].value_counts().head(10).keys())

netreturn_vis = netreturn_vis[netreturn_vis['genres'].isin(n_list)]

In [ ]:
# visualization 1: net return bar plot

# seaborn style
sns.set(style='whitegrid')

# initialize matlab plot
fig, ax = plt.subplots(figsize=(16, 8))

# define df, x and y values
roi_vis = moviebudgets_moviebasics_mean_df.sort_values(by='net_return', ascending=False).head(10)
x_values = list(roi_vis['genres'])
y_values = list(roi_vis['net_return'])

net_return_vis = moviebudgets_moviebasics_mean_df.sort_values(by='net_return', ascending=False).head(10)

sns.barplot(x=x_values, y=y_values, data=roi_vis, palette='muted')
ax.set_title('Genre vs Avg. Net Return', fontsize = 20, pad=24)
plt.xlabel('Genre', fontsize = 18, labelpad=18)
plt.ylabel('Net Return (Hundreds of Millions)', fontsize = 18, labelpad=18)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.savefig('./viz_images/' + 'net_return_visualization.png', bbox_inches='tight')

# Visualization 1 - Net Return Observations
This chart shows that the top 3 earning genres of film in terms of Net Return (Worldwide Gross - Production Budget) are Animation, Sci-Fi, and Musical/Adventure.

In addition, all of the top 20 Adventure films in terms of net return are sequels or have since produced sequels (Frozen, Zootopia) of existing, and already widely popular intellectual property (Marvel, James Bond, Jumanji, etc.). This informs our recommendation in that it suggests that the average return for a standalone adventure film is significantly less than what the average suggests. 

Since acquiring highly popular IP or producing sequels to existing IP is a difficult hurdle for a brand new studio to overcome, we do not recommend that the proposed Microsoft Film Studios produce an Action/Adventure blockbuster type of film as one of its first ventures into the industry.

In [ ]:
moviebudgets_moviebasics_df[moviebudgets_moviebasics_df['genres'] == 'Adventure'].sort_values(by='net_return', ascending=False).head(20)

In [ ]:
# visualization 2: ROI% bar plot

# seaborn style
sns.set(style='whitegrid')

# initialize matlab plot
fig, ax = plt.subplots(figsize=(16, 8))

# define df, x and y values
roi_vis = moviebudgets_moviebasics_mean_df.sort_values(by='roi_percent', ascending=False).head(10)
x_values = list(roi_vis['genres'])
y_values = list(roi_vis['roi_percent'])

# plot
sns.barplot(x=x_values, y=y_values,  data=roi_vis, palette='muted')
ax.set_title('Genre vs Avg. ROI', fontsize = 20, pad=24)
plt.xlabel('Film Genre', fontsize = 18, labelpad=18)
plt.ylabel('ROI %',  fontsize = 18, labelpad=18)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.savefig('./viz_images/' + 'roi_percent_visualization.png', bbox_inches='tight')

# Visualization 2 - Net Return Observations
Figure 2 shows that the genres with the highest ROI% are Mystery, Horror, and Thriller films with Mystery/Horror films returning 400% of the initial investment on average. 

This suggests that the production budgets for horror films on average are on the lower end of the spectrum, and offer studios the best bang for their buck. As such, our recommendation is that Microsoft Studios start out by producing Horror movies with relatively lower production budgets as a way to manage risk and maximize ROI%.

In [ ]:
moviebudgets_moviebasics_df[moviebudgets_moviebasics_df['genres'] == 'Horror'].sort_values(by='net_return', ascending=False).head(20)

# TN Movie Budgets Clean Up Process (Release Date Analysis)

In [ ]:

tn_movies = pd.read_csv('./zippedData/tn.movie_budgets.csv.gz')
                                
#Get rid of punctuation on worldwide_gross column
tn_movies['worldwide_gross'] = tn_movies['worldwide_gross'].replace('[\$,]', '', regex=True)

#Get rid of punctuation on domestic_gross column
tn_movies['domestic_gross'] = tn_movies['domestic_gross'].replace('[\$,]', '', regex=True)

#Get rid of punctuation on production_budget column
tn_movies['production_budget'] = tn_movies['production_budget'].replace('[\$,]', '', regex=True)

#change worldwide_gross column to float
tn_movies['worldwide_gross'] = tn_movies['worldwide_gross'].astype(float)

#change domestic_gross column to int
tn_movies['domestic_gross'] = tn_movies['domestic_gross'].astype(int)

#change production_budget column to int
tn_movies['production_budget']=tn_movies['production_budget'].astype(int)

In [ ]:
tn_movies.info()

As shown above, all of the fields are now the correct data type.

# Recommendation 2: Release Month Analysis

With this recommendation we decided to use the return on investment(ROI) percentage to see if there was a trend in getting more ROI from releasing a movie in a certain month.
The process that was used to get this percentage was taking the worldwide gross column minus production budget to get net income, then dividing net income by production budget to end up with an ROI percentage

In [ ]:
#create a net_income column by taking domestic_gross and subtracting production budget
tn_movies["net_return"]=tn_movies['worldwide_gross']-tn_movies['production_budget']

#create a cost_revenue_ratio by taking net income and dividing it by production budget
tn_movies["ROI"]=((tn_movies['net_return'])/(tn_movies['production_budget']))*100

#change all cost_rev values into int 
tn_movies["ROI"] =tn_movies["ROI"].astype(int)

#change release_date column into datetime format
tn_movies["release_date"] = pd.to_datetime(tn_movies["release_date"])

#create a month and year column
tn_movies['year'] = pd.DatetimeIndex(tn_movies['release_date']).year
tn_movies['month'] = pd.DatetimeIndex(tn_movies['release_date']).month

# time frame: 2010-Present & SAG definition of theatrical budget: > 2 million
tn_movies_2010 = tn_movies[(tn_movies['production_budget'] > 1999999) & (tn_movies['year'] >= 2010)]

In [ ]:
tn_movies_2010.info()

Our sample size is 1851 unique films released since 2010 that also has a production budget of $2 million or more (SAG definition of a theatrical movie). Net Return and ROI were added to allow for meaningful comparison among the films

# Recommendation 2: Month Release Visualization

In [ ]:
#created a list with all the Cost_Rev_Ratios in tn_movies_2010
ROI = [x for x in tn_movies_2010["ROI"]] 
#created a list of all the month value associated with each cost_rev_ratio
Month = [x for x in tn_movies_2010['month']]
#mapped both list into a master list as tuples
ROI_Month = list(map(lambda *x: x, Month, ROI))
#created a function that would take in associated number and append a new list of Cost_Rev_Ratio based on associated month value
#get the mean of the list by associated month  
def month_mean(num):
    x = []
    for index, tuple in enumerate(ROI_Month):
        element_one = tuple[0]
        element_two = tuple[1]
        if element_one == num:
            x.append(element_two)
    return(st.mean(x))
#created a function that would take in associated number and append a new list of Cost_Rev_Ratio based on associated month value
#get the median of the list by associated month 
def month_median(num):
    x = []
    for index, tuple in enumerate(ROI_Month):
        element_one = tuple[0]
        element_two = tuple[1]
        if element_one == num:
            x.append(element_two)
    return(st.median(x))

In [ ]:
#created dictionary to map month_index to month_list
month_index = [1,2,3,4,5,6,7,8,9,10,11,12]
month_list = ['Jan','Feb','Mar','Apr','May','June','July','Aug','Sept','Oct','Nov','Dec']
mapped_months = dict(map(lambda *x: x, month_index, month_list))

In [ ]:
#made two list using functions above of all medians per month and means per month 
by_month_mean_list = []
by_month_median_list = []
for num in mapped_months.keys():
    by_month_mean_list.append(month_mean(num))
    by_month_median_list.append(month_median(num))

In [ ]:
#created two dictionaries that mapped values of the two list above to associated month 
month_mean = dict(map(lambda *x: x, month_list, by_month_mean_list))
month_median = dict(map(lambda *x: x, month_list, by_month_median_list))

In [ ]:
month_mean

In [ ]:
month_median

The medians are lower than the averages, which shows a positive skew of the data. This makes sense as the average returns are likely to be pulled upward by the top earning films that outearn the average film by several magnitudes.

July has the highest average and also the highest median, suggesting that it is likely to be the most optimal month for releasing a movie.

In [ ]:
df = pd.DataFrame({'Month': Month,
                   'ROI': ROI,})
#set seaborn plotting aesthetics
sns.set(style='white')


ax = sns.barplot(x='Month', y='ROI',  data=df)
sns.set(rc={'figure.figsize':(11.7,8.27)})
ax.set_ylabel('ROI percentage',fontsize = 16)
ax.set_xlabel('Month',fontsize = 16)
ax.set_title('ROI by Month of Entire Cleaned Data',fontsize = 20)
ax.set_xticklabels(month_list)
plt.savefig('./viz_images/' + 'fulldata_ROI_month.png', bbox_inches='tight');

This chart supports our assumption that July would be the best month to release a movie as even when adjusting for outliers, July still leads all months by ROI percentage.

In [ ]:
df2 = pd.DataFrame({'Month': month_median.keys(),
                   'ROI': month_median.values(),})
#set seaborn plotting aesthetics
sns.set(style='white')

#create grouped bar chart
ax = sns.barplot(x='Month', y='ROI',  data=df2, color='blue')
sns.set(rc={'figure.figsize':(11.7,8.27)})
ax.set_ylabel('ROI percentage',fontsize = 16)
ax.set_xlabel('Month',fontsize = 16)
ax.set_title('ROI Median by Month',fontsize = 20)
plt.savefig('./viz_images/' + 'ROI_median_by_month.png', bbox_inches='tight');

In [ ]:
df1 = pd.DataFrame({'Month': month_mean.keys(),
                   'ROI': month_mean.values(),})
#set seaborn plotting aesthetics
sns.set(style='white')

#create grouped bar chart
ax = sns.barplot(x='Month', y='ROI',  data=df1,color='red')
sns.set(rc={'figure.figsize':(11.7,8.27)})
ax.set_ylabel('ROI percentage',fontsize = 16)
ax.set_xlabel('Month',fontsize = 16)
ax.set_title('ROI Mean by Month',fontsize = 20)
plt.savefig('./viz_images/' + 'ROI_mean_by_month.png', bbox_inches='tight');

In [ ]:
ax = pd.concat({
    'Mean': df1.set_index('Month'), 'Median': df2.set_index('Month'),
}, axis=1).plot.bar()
ax.set_ylabel('ROI percentage',fontsize = 16)
ax.set_xlabel('Month',fontsize = 16)
ax.set_title('ROI by Month',fontsize = 20)
plt.legend(labels=["Mean","Median"], fontsize = 14)
plt.savefig('./viz_images/' + 'ROI_combo_by_month', bbox_inches='tight');

# Recommendation 2: Analysis outcome 

Based on the different visualization trends we came to the conclusion that the best month of release would be July.

# IMDB Clean Up Process

In [ ]:
imdb_schema = pd.read_sql('''SELECT * FROM sqlite_master''',conn)

In [ ]:
writers_df = pd.read_sql('''select * from writers''', conn)
#writers kind of irrelevant for the question

In [ ]:
writers_df.describe()

In [ ]:
writers_df.info()

In [ ]:
principals_df = pd.read_sql('''select * from principals''', conn)
#useful for actor/actresses,, probably drop characters, job since job = none but category has actor

In [ ]:
principals_df['person_id'].value_counts()

In [ ]:
principals_df.describe()
#useless

In [ ]:
principals_df.info()
#Probably filter out characters since we want actors/actresses

In [ ]:
movie_basics_df = pd.read_sql('''select * from movie_basics''', conn)
movie_basics_df.head()
#Very useful, movie_Id common denominator

In [ ]:
movie_basics_df.info()
#Lots of nulls in RunTime_Minutes, quite a few in Genres and some in Original Title
#Could probably drop original title while cleaning the data

In [ ]:
movie_basics_df.describe()
#Max 2115? Probably have to put a max on the SQL query for it, same for runtime 

In [ ]:
directors_df = pd.read_sql("""select * from directors""",conn)
directors_df.head()
#Merge off of movie_id to main database
#Convert person_ID through persons_df
#Usable, directors are nice

In [ ]:
directors_df['person_id'].value_counts()

In [ ]:
directors_df.describe()
#nothing too relevant

In [ ]:
directors_df.info()
#complete data

In [ ]:
known_for_df = pd.read_sql("""select * from known_for""",conn)
known_for_df.head()
#I dont really think this data set is relevant - dropping it
#Based off data it seems to just be a person (would have to merge to person_df) and what movie theyre known for

In [ ]:
movie_akas_df = pd.read_sql("""select * from movie_akas""",conn)
movie_akas_df.head()
#Nothing too amazing, considering dropping this dataset

In [ ]:
movie_akas_df.info()
#Tons of nulls in language, type, and attributes; even region 

In [ ]:
movie_akas_df.describe()
#Useless

In [ ]:
movie_akas_df['movie_id'].value_counts()
#Tons of duplicates too based off movie id and how many regions released, probably irrelevant and too much information
#dropping this dataset too


In [ ]:
movie_ratings_df = pd.read_sql("""select * from movie_ratings""",conn)
movie_ratings_df.head()
#Pretty useful information

In [ ]:
movie_ratings_df.info()
#Full dataset, nice

In [ ]:
round(movie_ratings_df.describe(),2)
#Rating scale looks clean, numvotes looks fine too since no negatives

In [ ]:
movie_ratings_df['movie_id'].value_counts()
#No dupes

In [ ]:
persons_df = pd.read_sql("""select * from persons""",conn)
persons_df.head()
#Super useful for mapping person ID to primary names, rest of the information is a bit much
#Primary profession could be useful but can't be connected to actual movies since no movie_Id in dataset
#birth_year

In [ ]:
persons_df.describe()
#Probably remove dead people honestly

In [ ]:
persons_df.info()
#Tons of null in birth year and death year

In [ ]:
subdir = pd.read_sql(
'''
SELECT
    dir.movie_id,
    per.primary_name
FROM
    directors as dir
    LEFT JOIN persons as per
        ON dir.person_id = per.person_id
WHERE
    per.death_year is null
''',
conn
)
subdir.head()
#Filters out dead directors for movies 

In [ ]:
subdir.shape

In [ ]:
subpri = pd.read_sql(
'''
SELECT
    pri.movie_id,
    per.primary_name,
    pri.category
FROM
    principals as pri
    LEFT JOIN persons as per
        ON pri.person_id = per.person_id
WHERE
    (pri.category = 'actor'
    OR
    pri.category = 'actress'
    or
    pri.category = 'director')
    AND
    per.death_year is null
    AND
    per.primary_name is not null
''',
conn
)
subpri.head()
#Finding the actor/actresses, can be used to create a subquery
#director subquery potentially useless

In [ ]:
subpri.shape

In [ ]:
subpri['category'].value_counts()

In [ ]:
subpri.info()

In [ ]:
imdb = pd.read_sql(
'''
SELECT
    mb.primary_title as MovieName,
    mb.start_year as StartYear,
    mb.runtime_minutes as RunTime,
    mb.genres as Genre,
    mr.averagerating as Rating,
    mr.numvotes as VoteCount   
FROM
    movie_basics as mb
    LEFT JOIN movie_ratings as mr
        ON mb.movie_id = mr.movie_id
WHERE
    mr.numvotes > 1000
    AND
    averagerating is not null
    AND
    start_year >= 2010
    and 
    runtime_minutes is not null
    and
    genres is not null 
''',
conn
)
imdb.head()
#overall imdb data, could be good for finding average ratings, genres, etc on whats good

In [ ]:
imdb.shape
#9601 movies left

In [ ]:
imdb.info()
#no nulls pog

In [ ]:
imdb2 = imdb
#creating a new dataframe to not mess with the original

In [ ]:
imdb2['Genre'] = imdb['Genre'].str.split(',')
#String splitting genre for exploding purposes

In [ ]:
imdb2.head(5)
#check it

In [ ]:
imdb2['StartYear'].value_counts()
#Checking range of years for data

In [ ]:
imdbexp = imdb2.explode('Genre')
#explode to separate all the genres

In [ ]:
imdbexp['Genre'].value_counts()

In [ ]:
imdbexp.shape

In [ ]:
imdbexpg = imdbexp.loc[imdbexp['Genre'] == 'Animation'].sort_values('Rating',ascending=False)
imdbexpg.head(20)

In [ ]:
imdbexp.head(5)
#check it

In [ ]:
imdbexp.info()
#no nulls nice

In [ ]:
imdb_work = pd.read_sql(
'''
SELECT
    mb.start_year as MovieYear,
    mb.primary_title as MovieName,
    mr.averagerating as AvgRating,
    job.primary_name as Name,
    genres as Genre,
    job.category as Job
FROM
    (SELECT
        pri.movie_id,
        per.primary_name,
        pri.category
            FROM
        principals as pri
            LEFT JOIN persons as per
            ON pri.person_id = per.person_id
    WHERE
        (pri.category = 'actor'
        OR
        pri.category = 'actress'
        or
        pri.category = 'director')
        AND
        per.death_year is null
        AND
        per.primary_name is not null) as job
    LEFT JOIN movie_basics as mb
        ON job.movie_id = mb.movie_id
    LEFT JOIN movie_ratings as mr
        on job.movie_id = mr.movie_id
WHERE
    mr.numvotes > 1000
    AND
    averagerating is not null
    AND
    start_year >= 2010
    and 
    runtime_minutes is not null
    and
    genres is not null
ORDER BY
    job.primary_name,
    mb.start_year
''',
conn
)
imdb_work.head(10)
#This ensures actors or actresses with more than enough experience on an average rating
#this is only for names

In [ ]:
imdb_workaction = imdb_work.loc[imdb_work['Genre'].str.contains('Action')]

In [ ]:
imdb_workh = imdb_work.loc[imdb_work['Genre'].str.contains('Horror')]

In [ ]:
imdb_workaction.head(2)

In [ ]:
imdb_workaction.shape

In [ ]:
imdb_workh.head(2)

In [ ]:
imdb_workh.shape

In [ ]:
imdb_work.loc[imdb_work['Name'] == '50 Cent']['MovieYear'].corr(imdb_work.loc[imdb_work['Name'] == '50 Cent']['AvgRating'])
#testing code for correlation

In [ ]:
imdb_names = pd.read_sql(
'''
SELECT
    job.primary_name as Name,
    job.category as Job,
    genres as Genre,
    AVG(mr.averagerating) as Rating
FROM
    (SELECT
        pri.movie_id,
        per.primary_name,
        pri.category
            FROM
        principals as pri
            LEFT JOIN persons as per
            ON pri.person_id = per.person_id
    WHERE
        (pri.category = 'actor'
        OR
        pri.category = 'actress'
        or
        pri.category = 'director')
        AND
        per.death_year is null
        AND
        per.primary_name is not null) as job
    LEFT JOIN movie_basics as mb
        ON job.movie_id = mb.movie_id
    LEFT JOIN movie_ratings as mr
        on job.movie_id = mr.movie_id
WHERE
    mr.numvotes > 1000
    AND
    averagerating is not null
    AND
    start_year >= 2010
    and 
    runtime_minutes is not null
    and
    genres is not null
GROUP BY
    job.primary_name
HAVING
    count(mr.averagerating) > 4
ORDER BY
    AVG(mr.averagerating) DESC
''',
conn
)
imdb_names.head()
#This ensures actors or actresses with more than enough experience on an average rating
#this is only for names for all genres with a count above 5

In [ ]:
imdb_namesaction = pd.read_sql(
'''
SELECT
    job.primary_name as Name,
    job.category as Job,
    genres as Genre,
    AVG(mr.averagerating) as Rating
FROM
    (SELECT
        pri.movie_id,
        per.primary_name,
        pri.category
            FROM
        principals as pri
            LEFT JOIN persons as per
            ON pri.person_id = per.person_id
    WHERE
        (pri.category = 'actor'
        OR
        pri.category = 'actress'
        or
        pri.category = 'director')
        AND
        per.death_year is null
        AND
        per.primary_name is not null) as job
    LEFT JOIN movie_basics as mb
        ON job.movie_id = mb.movie_id
    LEFT JOIN movie_ratings as mr
        on job.movie_id = mr.movie_id
WHERE
    mr.numvotes > 1000
    AND
    averagerating is not null
    AND
    start_year >= 2010
    and 
    runtime_minutes is not null
    and
    genres like '%Action%'
GROUP BY
    job.primary_name
HAVING
    count(mr.averagerating) > 4
ORDER BY
    AVG(mr.averagerating) DESC
''',
conn
)
imdb_namesaction.head()
#This ensures actors or actresses with more than enough experience on an average rating
#this is only for names, this is for action only

In [ ]:
imdb_namesh = pd.read_sql(
'''
SELECT
    job.primary_name as Name,
    job.category as Job,
    genres as Genre,
    AVG(mr.averagerating) as Rating
FROM
    (SELECT
        pri.movie_id,
        per.primary_name,
        pri.category
            FROM
        principals as pri
            LEFT JOIN persons as per
            ON pri.person_id = per.person_id
    WHERE
        (pri.category = 'actor'
        OR
        pri.category = 'actress'
        or
        pri.category = 'director')
        AND
        per.death_year is null
        AND
        per.primary_name is not null) as job
    LEFT JOIN movie_basics as mb
        ON job.movie_id = mb.movie_id
    LEFT JOIN movie_ratings as mr
        on job.movie_id = mr.movie_id
WHERE
    mr.numvotes > 1000
    AND
    averagerating is not null
    AND
    start_year >= 2010
    and 
    runtime_minutes is not null
    and
    genres like '%Horror%'
GROUP BY
    job.primary_name
HAVING
    count(mr.averagerating) > 3
ORDER BY
    AVG(mr.averagerating) DESC
''',
conn
)
imdb_namesh.head()
#This ensures actors or actresses with more than enough experience on an average rating
#this is only for names, this is for horror with a count above 3

In [ ]:
imdb_namesh.loc[imdb_namesh['Job']=='actor'].shape
#Usable

In [ ]:
imdb_namesh.loc[imdb_namesh['Job']=='actress'].shape
#Usable

In [ ]:
imdb_namesh.loc[imdb_namesh['Job']=='director'].shape
#Usable

In [ ]:
imdb_namesh.shape
#Usable

In [ ]:
imdb_namesaction.shape
#Usable

In [ ]:
imdb_names.head(2)
#Nice

In [ ]:
imdb_actorsh = imdb_namesh.loc[imdb_namesh['Job']=='actor'].reset_index(drop=True)
imdb_actorsh.head(2) #Actors only

In [ ]:
actors_corrh = pd.DataFrame({'Correlation':[imdb_workh.loc[imdb_workh['Name'] == actor]['MovieYear'].corr(imdb_workh.loc[imdb_workh['Name'] == actor]['AvgRating']) for actor in imdb_actorsh['Name']]})
#creating correlation series for the actors with prior code

In [ ]:
imdb_actorsh['Correlation'] = actors_corrh
imdb_actorsh.head(2)
#adding correlation to it

In [ ]:
imdb_actorsh.info() #nice

In [ ]:
imdb_actressh = imdb_namesh.loc[imdb_namesh['Job']=='actress'].reset_index(drop=True)
imdb_actressh.head(2) #Actress only

In [ ]:
actress_corrh = pd.DataFrame({'Correlation':[imdb_workh.loc[imdb_workh['Name'] == actress]['MovieYear'].corr(imdb_workh.loc[imdb_workh['Name'] == actress]['AvgRating']) for actress in imdb_actressh['Name']]})
actress_corrh.head(2) #Correlation for females

In [ ]:
imdb_actressh['Correlation'] = actress_corrh
#Adding the correlation column to imdb actress

In [ ]:
imdb_actressh.head(2) #Checking new data with corr

In [ ]:
imdb_actressh.info() #All is well

In [ ]:
imdb_directorh = imdb_namesh.loc[imdb_namesh['Job']=='director'].reset_index(drop=True)
imdb_directorh.head(10) #Director

In [ ]:
director_corrh = pd.DataFrame({'Correlation':[imdb_workh.loc[imdb_workh['Name'] == director]['MovieYear'].corr(imdb_workh.loc[imdb_workh['Name'] == director]['AvgRating']) for director in imdb_directorh['Name']]})
director_corrh.head(2)  #Creating correlation for directors now

In [ ]:
imdb_directorh['Correlation'] = director_corrh #putting the correlation into the directors

In [ ]:
imdb_directorh.head(2) #All is well

In [ ]:
imdb_directorh.info() #All is well, only 85 directors weird

In [ ]:
scatact = sns.scatterplot(x='Rating',y='Correlation',hue='Name',data=imdb_actorsh.head(10), marker = '*', s=1000)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.suptitle('Actors Relationship to Horror Movie Rating',fontsize=30)
plt.xlabel('Average Movie Rating',fontsize=20)
plt.ylabel('Correlation of Movie Rating from 2010-2019',fontsize=20)
plt.savefig('./viz_images/' + 'Actor Relationship to Horror Movie Rating', bbox_inches='tight')
#used this to pull people I like, removed from presentation cause its useless for executives to know

In [ ]:
scatact = sns.scatterplot(x='Rating',y='Correlation',hue='Name',data=imdb_actressh.head(10), marker = '*', s=1000)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.suptitle('Actresses Relationship to Horror Movie Rating',fontsize=30)
plt.xlabel('Average Movie Rating',fontsize=20)
plt.ylabel('Correlation of Movie Rating from 2010-2019',fontsize=20)
plt.savefig('./viz_images/' + 'Actresses Relationship to Horror Movie Rating.png', bbox_inches='tight')
#used this to pull people I like, removed from presentation cause its useless for executives to know

In [ ]:
scatact = sns.scatterplot(x='Rating',y='Correlation',hue='Name',data=imdb_directorh.head(10), marker = '*', s=1000)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.suptitle('Directors Relationship to Horror Movie Rating',fontsize=30)
plt.xlabel('Average Movie Rating',fontsize=20)
plt.ylabel('Correlation of Movie Rating from 2010-2019',fontsize=20)
plt.savefig('./viz_images/' + 'Directors Relationship to Horror Movie Rating', bbox_inches='tight')
#used this to pull people I like, removed from presentation cause its useless for executives to know

In [ ]:
logan = imdb_workh.loc[imdb_workh['Name']=='Logan Miller'].groupby('MovieYear').mean()
patrick = imdb_workh.loc[imdb_workh['Name']=='Patrick Wilson'].groupby('MovieYear').mean()
joe = imdb_workh.loc[imdb_workh['Name']=='Joe Swanberg'].groupby('MovieYear').mean()
#Setting up infrastructure for the chart
#Used to groupby and average out movie ratings from the same year
x = np.linspace(0, 10, 1000)
fig, ax = plt.subplots(figsize=(10, 4))
ax.plot(logan.index.values, 'AvgRating', data=logan, label='Logan Miller')
ax.plot(patrick.index.values, 'AvgRating', data=patrick, label='Patrick Wilson')
ax.plot(joe.index.values, 'AvgRating', data=joe, label='Joe Swanberg')
ax.set_xlabel('Year')
ax.set_ylabel('Movie Average Rating')
ax.set_title('Horror Film Rating vs Year by Actors', fontsize=20)
loc='bottom right'
plt.tight_layout()
ax.legend(loc='lower right', title='Actors')
plt.savefig('./viz_images/' + 'Horror Film Rating vs Years by Actors', bbox_inches='tight')
#Line chart for directors, recommend Logan, Patrick, Joe

In [ ]:
amy = imdb_workh.loc[imdb_workh['Name']=='Amy Seimetz'].groupby('MovieYear').mean()
kate = imdb_workh.loc[imdb_workh['Name']=='Kate Bosworth'].groupby('MovieYear').mean()
danielle = imdb_workh.loc[imdb_workh['Name']=='Danielle Harris'].groupby('MovieYear').mean()
#Setting up infrastructure for the chart
#Used to groupby and average out movie ratings from the same year
x = np.linspace(0, 10, 1000)
fig, ax = plt.subplots(figsize=(10, 4))
ax.plot(amy.index.values, 'AvgRating', data=amy, label='Amy Seimetz')
ax.plot(kate.index.values, 'AvgRating', data=kate, label='Kate Bosworth')
ax.plot(danielle.index.values, 'AvgRating', data=danielle, label='Danielle Harris')
ax.set_xlabel('Year')
ax.set_ylabel('Movie Average Rating')
ax.set_title('Horror Film Rating vs Year by Actress',fontsize=20)
ax.legend(loc='lower right', title='Actress')
plt.tight_layout()
plt.savefig('./viz_images/' + 'Horror Film Rating vs Year by Actress', bbox_inches='tight')
#Line chart for actress, recommend Amy, Kate, Danielle Harris

In [ ]:
alper = imdb_workh.loc[imdb_workh['Name']=='Alper Mestçi'].groupby('MovieYear').mean()
james = imdb_workh.loc[imdb_workh['Name']=='James Wan'].groupby('MovieYear').mean()
mike = imdb_workh.loc[imdb_workh['Name']=='Mike Flanagan'].groupby('MovieYear').mean()
takashi = imdb_workh.loc[imdb_workh['Name']=='Takashi Miike'].groupby('MovieYear').mean()
#Setting up infrastructure for the chart
#Used to groupby and average out movie ratings from the same year
x = np.linspace(0, 10, 1000)
fig, ax = plt.subplots(figsize=(10, 4))
ax.plot(alper.index.values, 'AvgRating', data=alper, label='Alper Mestci')
ax.plot(james.index.values, 'AvgRating', data=james, label='James Wan')
ax.plot(mike.index.values, 'AvgRating', data=mike, label='Mike Flanagan') 
ax.set_xlabel('Year')
ax.set_ylabel('Movie Average Rating')
ax.set_title('Horror Film Rating vs Year by Director',fontsize=20)
plt.tight_layout()
ax.legend(loc='lower right', title='Directors')
plt.savefig('./viz_images/' + 'Horror Film Rating vs Year by Director', bbox_inches='tight')
#Line chart for Actors recommend Alper, James, Mike

In [ ]:
imdb_workwork = imdb_workh.loc[imdb_workh['Name'].str.contains('Joe Swanberg')]
imdb_workwork
#Testing code, should have just used workwork instead of the one below

In [ ]:
imdb_work.loc[imdb_work['Genre'].str.contains('Horror')].groupby('Genre')['AvgRating'].mean().mean()
#Good to know th emean average of horror movies

In [ ]:
imdb_workh.loc[imdb_workh['Name'] == 'Alper Mestçi']
#checking the titles to use as information source on who they are